# ATIVIDADE 1 - MULTIPLICAÇÃO DE MATRIZES EM CUDA

### INSTALANDO O PYCUDA


In [1]:
!pip install pycuda


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.4 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660425 sha256=d50f5a2a4b92f6729ab8e9114f9dd19c600575ed2a280ba5a78e5578ab595ebc
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


### Importações e geração das matrizes

In [29]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import time

# Dimensões da matriz
M, K, N = 512, 512, 512

# Matrizes de entrada e saída
A = np.random.randint(0, 10, (M, K)).astype(np.int32)
B = np.random.randint(0, 10, (K, N)).astype(np.int32)
C = np.zeros((M, N), dtype=np.int32)


### PRINT DAS MATRIZES GERADAS

In [30]:
print("Matriz A:")
print(A)

print("\nMatriz B:")
print(B)

print("\nResultado da multiplicação na GPU (Matriz C):")
print(C)


Matriz A:
[[3 0 4 ... 1 1 9]
 [5 6 6 ... 7 3 3]
 [7 0 0 ... 1 2 8]
 ...
 [4 2 3 ... 5 2 8]
 [9 2 4 ... 2 8 4]
 [0 8 5 ... 1 9 1]]

Matriz B:
[[6 4 0 ... 6 1 3]
 [5 7 3 ... 6 5 4]
 [3 5 5 ... 7 5 9]
 ...
 [8 4 5 ... 0 5 8]
 [9 0 5 ... 4 3 8]
 [6 2 5 ... 1 8 0]]

Resultado da multiplicação na GPU (Matriz C):
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### MULTIPLICAÇÃO NA CPU USANDO NUMPY

In [31]:
start_time_cpu = time.time()
C_cpu = np.dot(A, B)
end_time_cpu = time.time()

print(f"Tempo de execução na CPU: {end_time_cpu - start_time_cpu:.6f} segundos")


Tempo de execução na CPU: 0.244950 segundos


### Definindo bloco e kernel CUDA com PyCUDA

In [32]:
BLOCK_SIZE = 16

kernel_code = """
__global__ void MatrixMulKernel(int *A, int *B, int *C, int M, int N, int K) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < N) {
        int temp = 0;
        for (int i = 0; i < K; ++i) {
            temp += A[row * K + i] * B[i * N + col];
        }
        C[row * N + col] = temp;
    }
}
"""


### Compilação do kernel e alocação na GPU

In [33]:
mod = SourceModule(kernel_code)
matrixmul = mod.get_function("MatrixMulKernel")

# Alocação de memória e cópia dos dados para GPU
A_gpu = cuda.mem_alloc(A.nbytes)
B_gpu = cuda.mem_alloc(B.nbytes)
C_gpu = cuda.mem_alloc(C.nbytes)

cuda.memcpy_htod(A_gpu, A)
cuda.memcpy_htod(B_gpu, B)


### Execução na GPU

In [34]:
grid = ((N + BLOCK_SIZE - 1) // BLOCK_SIZE, (M + BLOCK_SIZE - 1) // BLOCK_SIZE)
block = (BLOCK_SIZE, BLOCK_SIZE, 1)

start_time_gpu_16 = time.time()
matrixmul(A_gpu, B_gpu, C_gpu, np.int32(M), np.int32(N), np.int32(K), block=block, grid=grid)
cuda.memcpy_dtoh(C, C_gpu)
end_time_gpu_16 = time.time()

print(f"Tempo de execução na GPU: {end_time_gpu - start_time_gpu:.6f} segundos")


Tempo de execução na GPU: 0.001866 segundos


### COMPARAÇÃO DOS RESULTADOS ENTRE CPU E BLOCO DE TAMANHO 16

In [35]:
if np.allclose(C, C_cpu):
    print("Resultado CORRETO (GPU == CPU)")
else:
    print("Resultado INCORRETO (diferença entre GPU e CPU)")


Resultado CORRETO (GPU == CPU)


### Comparação dos tempos

In [36]:
tempo_cpu = end_time_cpu - start_time_cpu
tempo_gpu_16 = end_time_gpu_16 - start_time_gpu_16

print("\n====== COMPARAÇÃO DE DESEMPENHO ======")
if tempo_cpu < tempo_gpu_16:
    print(f"A CPU foi mais rápida por {tempo_gpu_16 - tempo_cpu:.6f} segundos.")
else:
    print(f"A GPU foi mais rápida por {tempo_cpu - tempo_gpu_16:.6f} segundos.")

print(f"\nTempo CPU: {tempo_gpu_16:.6f} segundos")
print(f"Tempo GPU: {tempo_gpu_16:.6f} segundos")



====== COMPARAÇÃO DE DESEMPENHO ======
A GPU foi mais rápida por 0.243045 segundos.

Tempo CPU: 0.001905 segundos
Tempo GPU: 0.001905 segundos


### PARA BLOCO DE TAMANHO 32

In [37]:
BLOCK_SIZE = 32  # novo tamanho de bloco

# Recalcula grid e block
block = (BLOCK_SIZE, BLOCK_SIZE, 1)
grid = ((N + BLOCK_SIZE - 1) // BLOCK_SIZE, (M + BLOCK_SIZE - 1) // BLOCK_SIZE)

start_time_gpu_32 = time.time()
matrixmul(A_gpu, B_gpu, C_gpu, np.int32(M), np.int32(N), np.int32(K), block=block, grid=grid)
cuda.memcpy_dtoh(C, C_gpu)
end_time_gpu_32 = time.time()

print(f"GPU com BLOCK_SIZE=32: {end_time_gpu_32 - start_time_gpu_32:.6f} segundos")

GPU com BLOCK_SIZE=32: 0.001577 segundos


### COMPARAÇÃO DOS RESULTADOS ENTRE CPU E BLOCO DE TAMANHO 32

In [38]:
if np.allclose(C, C_cpu):
    print("Resultado CORRETO (GPU == CPU)")
else:
    print("Resultado INCORRETO (diferença entre GPU e CPU)")


Resultado CORRETO (GPU == CPU)


### COMPARAÇÃO DE TEMPO ENTRE BLOCO DE 32 E 16

In [39]:
tempo_gpu_16 = end_time_gpu_16 - start_time_gpu_16
tempo_gpu_32 = end_time_gpu_32 - start_time_gpu_32

print("\n====== COMPARAÇÃO ENTRE BLOCK_SIZES ======")
if tempo_gpu_16 < tempo_gpu_32:
    print(f"BLOCO 16x16 foi mais rápido por {tempo_gpu_32 - tempo_gpu_16:.6f} segundos")
else:
    print(f"BLOCO 32x32 foi mais rápido por {tempo_gpu_16 - tempo_gpu_32:.6f} segundos")



====== COMPARAÇÃO ENTRE BLOCK_SIZES ======
BLOCO 32x32 foi mais rápido por 0.000328 segundos
